In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pylab as plt


def empty(a):
    pass

In [ ]:
#funtion to mask out 'region of interest'
def ROI(image, points):
    #creating similar size 'black' image
    mask = np.zeros_like(image)
    #fitting the point of polygon on the mask
    cv.fillPoly(mask, points, (255,255,255))
    #combining 'masked image' and 'original image'
    masked_image = cv.bitwise_and(image, mask)

    return masked_image
#function to draw lines on the image by Hough Transform
def drawLine(image, lines):
    #making an copy of the original image
    img = np.copy(image)
    #making the copied image of pixel values 0
    base = np.zeros((img.shape[0], img.shape[1],3), np.uint8)
    #Line from Hough Transform
    for line in lines:
        #Two end co-ordinate of the line
        for ax,ay, bx, by in line:
            #creating the line out of the co-ordinate
            cv.line(base, (ax,ay), (bx,by), (255,0,255), 5)
    #merging original image and lined image
    img = cv.addWeighted(img, 0.5, base, 1, 0.0)

    return img

In [ ]:
def mainProcess(image):
    #create grayscale version of original image
    imgGray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    #Blurring image with Gaussian image
    imgBlur = cv.GaussianBlur(imgGray,(7,7),0)
    #Edge detection by canny method
    imgCanny = cv.Canny(imgBlur,40,100)
    #custom polygon mask for ROI
    polygon2 = np.array([[(220,160),(0,240),(0,410),(280,160),(380,160),(640,410),(640,240),(440,160)]])
    #def ROI function
    mask = ROI(imgCanny, polygon2)
    #producing lines over 'masked image' using HoughP Transform
    HPline = cv.HoughLinesP(mask, rho=3, theta=np.pi/60, threshold=30, lines=np.array([]), minLineLength=70, maxLineGap=20)
    #def drawLine function
    result = drawLine(image, HPline)
    return result

#reading video
cap = cv.VideoCapture("taskResources/lane_vgt.mp4")
while(cap.isOpened()):
    #reading each frame
    sucess, frame = cap.read()
    #MAIN PROCESSING
    final = mainProcess(frame)
    #result frame shown i.e video shown
    cv.imshow("Processed",final)
    if cv.waitKey(5) & 0xFF == ord('q'):
        print("INTERRUPTED")
        break

In [ ]:
cap.release()
cv.destroyAllWindows()